In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow_addons import losses
from sklearn.metrics import confusion_matrix , classification_report

%matplotlib inline

In [ ]:
pip install tensorflow_addons

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('./customer_churn.csv')
df.sample(5)

In [ ]:
df.drop('customerID', axis='columns', inplace=True)

In [ ]:
df.columns

In [ ]:
df.dtypes

# Check for any null values and change TotalCharges to number

In [ ]:
pd.to_numeric(df.TotalCharges, errors='coerce').isnull()

In [ ]:
df[pd.to_numeric(df.TotalCharges,errors='coerce').isnull()]


In [ ]:
df.shape

# Get a data frame whitout TotalCharges null 

In [ ]:
df1 = df[df.TotalCharges!=' ']

In [ ]:
df1.shape

# TotalCharges from objec to numberic 


In [ ]:
df1.TotalCharges = pd.to_numeric(df1.TotalCharges)

In [ ]:
df1.TotalCharges.values

# Funtion to print columns and uniqueue values in it

In [ ]:
def print_unique_col_values(df):
    for column in df:
        print(f"{column} : {df[column].unique()}")

In [ ]:
print_unique_col_values(df1)

# Replace some values to simple No 

In [ ]:
df1.replace('No internet service', 'No', inplace=True)
df1.replace('No phone service', 'No', inplace=True)

In [ ]:
print_unique_col_values(df1)

# Replace Yes No to 1 and 0

In [ ]:
yes_no_columns = ['Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'OnlineSecurity',
                 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
                 'PaperlessBilling', 'Churn']

for col in yes_no_columns:
    df1.replace({'Yes': 1, 'No': 0} , inplace=True)

df1['gender'].replace({'Female': 1, 'Male': 0}, inplace=True)

# DO one hot encoding for the columns having differnet values

In [ ]:
df2 = pd.get_dummies(data=df1, columns=['InternetService', 'Contract', 'PaymentMethod'])

In [ ]:
df2.columns

In [ ]:
df2.sample(10)

# convert boolean type columns to numeric 

In [ ]:
boolean_to_numeric = ['InternetService_0', 'InternetService_DSL', 'InternetService_Fiber optic',
                     'Contract_Month-to-month', 'Contract_One year', 'Contract_Two year', 'PaymentMethod_Bank transfer (automatic)',
                       'PaymentMethod_Credit card (automatic)', 'PaymentMethod_Electronic check', 'PaymentMethod_Mailed check']

for bl in boolean_to_numeric: 
    df2[bl] = df2[bl].astype(int)

# Scale the values from 0 to 1 in following columns

In [ ]:
df2.columns

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

col_to_scale = ['tenure', 'MonthlyCharges', 'TotalCharges']
df2[col_to_scale] = scaler.fit_transform(df2[col_to_scale])

In [ ]:
print_unique_col_values(df2)

# Create a function to build ANN model with tensorflow/keras

In [ ]:
def ANN(X_train, y_train, X_test, y_test, loss, weights):
    model = keras.Sequential([
        keras.layers.Dense(26, input_dim=26, activation='relu'),
        keras.layers.Dense(15, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
    
    if weights == -1:
        model.fit(X_train, y_train, epochs=100)
    else:
        model.fit(X_train, y_train, epochs=100, class_weight = weights)
    
    print(model.evaluate(X_test, y_test))
    
    y_preds = model.predict(X_test)
    y_preds = np.round(y_preds)
    
    print("Classification Report: \n", classification_report(y_test, y_preds))
    
    return y_preds

# Method1 : Undersampling

In [ ]:
count_class_0_ma, count_class_1_mi = df2.Churn.value_counts()
count_class_0_ma, count_class_1_mi

In [ ]:
# divide the df2 by Churn 1 and 0 
df2_class_0_ma = df2[df2['Churn'] == 0]
df2_class_1_mi = df2[df2['Churn'] == 1]

In [ ]:
# unsersample majority calls that is df2_class_0_ma then combine that with the minority class to form the balanced df

df2_class_0_under = df2_class_0_ma.sample(count_class_1_mi)
df2_undersampled = pd.concat([df2_class_0_under, df2_class_1_mi], axis=0)

In [ ]:
df2_undersampled.Churn.value_counts()

In [ ]:
X = df2_undersampled.drop('Churn', axis='columns')
y = df2_undersampled['Churn']

In [ ]:
y.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)

In [132]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
259/259 [==============================] - 1s 1ms/step - loss: 0.5383 - accuracy: 0.7393
Epoch 2/100
259/259 [==============================] - 0s 915us/step - loss: 0.4743 - accuracy: 0.7734
Epoch 3/100
259/259 [==============================] - 0s 855us/step - loss: 0.4631 - accuracy: 0.7777
Epoch 4/100
259/259 [==============================] - 0s 1ms/step - loss: 0.4574 - accuracy: 0.7814
Epoch 5/100
259/259 [==============================] - 0s 973us/step - loss: 0.4525 - accuracy: 0.7849
Epoch 6/100
259/259 [==============================] - 0s 884us/step - loss: 0.4478 - accuracy: 0.7861
Epoch 7/100
259/259 [==============================] - 0s 909us/step - loss: 0.4434 - accuracy: 0.7903
Epoch 8/100
259/259 [==============================] - 0s 903us/step - loss: 0.4386 - accuracy: 0.7921
Epoch 9/100
259/259 [==============================] - 0s 1ms/step - loss: 0.4342 - accuracy: 0.7959
Epoch 10/100
259/259 [==============================] - 0s 1ms/step - loss: 0.4

# Method2: Oversampling 

In [ ]:
df2_class_1_mi_over = df2_class_1_mi.sample(count_class_0_ma, replace=True)
df2_oversampled = pd.concat([df2_class_0_ma, df2_class_1_mi_over], axis=0)

In [ ]:
df2_oversampled.Churn.value_counts()

In [ ]:
df2_oversampled.shape

In [ ]:
X = df2_oversampled.drop('Churn', axis='columns')
y = df2_oversampled['Churn']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)

In [131]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
259/259 [==============================] - 1s 886us/step - loss: 0.5058 - accuracy: 0.7669
Epoch 2/100
259/259 [==============================] - 0s 898us/step - loss: 0.4687 - accuracy: 0.7814
Epoch 3/100
259/259 [==============================] - 0s 918us/step - loss: 0.4611 - accuracy: 0.7822
Epoch 4/100
259/259 [==============================] - 0s 1ms/step - loss: 0.4558 - accuracy: 0.7827
Epoch 5/100
259/259 [==============================] - 0s 970us/step - loss: 0.4513 - accuracy: 0.7849
Epoch 6/100
259/259 [==============================] - 0s 902us/step - loss: 0.4478 - accuracy: 0.7868
Epoch 7/100
259/259 [==============================] - 0s 885us/step - loss: 0.4440 - accuracy: 0.7921
Epoch 8/100
259/259 [==============================] - 0s 885us/step - loss: 0.4435 - accuracy: 0.7890
Epoch 9/100
259/259 [==============================] - 0s 921us/step - loss: 0.4377 - accuracy: 0.7959
Epoch 10/100
259/259 [==============================] - 0s 869us/step - los

# Method3: SMOTE

## Install pip install imbalanced-learn 

In [104]:
X = df2.drop('Churn', axis='columns')
y = df2['Churn']

In [114]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='minority')
X_sm, y_sm = smote.fit_resample(X, y)

In [119]:
y_sm.value_counts()

Churn
0    5163
1    5163
Name: count, dtype: int64

In [121]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.2, random_state=15, stratify=y_sm)

In [129]:
y_test.value_counts()

Churn
1    1033
0    1033
Name: count, dtype: int64

In [130]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
259/259 [==============================] - 1s 1ms/step - loss: 0.5584 - accuracy: 0.7217
Epoch 2/100
259/259 [==============================] - 0s 905us/step - loss: 0.4755 - accuracy: 0.7763
Epoch 3/100
259/259 [==============================] - 0s 921us/step - loss: 0.4649 - accuracy: 0.7797
Epoch 4/100
259/259 [==============================] - 0s 957us/step - loss: 0.4576 - accuracy: 0.7855
Epoch 5/100
259/259 [==============================] - 0s 897us/step - loss: 0.4522 - accuracy: 0.7854
Epoch 6/100
259/259 [==============================] - 0s 809us/step - loss: 0.4470 - accuracy: 0.7924
Epoch 7/100
259/259 [==============================] - 0s 887us/step - loss: 0.4420 - accuracy: 0.7929
Epoch 8/100
259/259 [==============================] - 0s 780us/step - loss: 0.4381 - accuracy: 0.7947
Epoch 9/100
259/259 [==============================] - 0s 912us/step - loss: 0.4337 - accuracy: 0.7982
Epoch 10/100
259/259 [==============================] - 0s 789us/step - los

# Method4: Use of Ensemble with undersampling

In [134]:
df2.Churn.value_counts()

Churn
0    5163
1    1869
Name: count, dtype: int64

In [136]:
X = df2.drop('Churn', axis='columns')
y = df2['Churn']

In [140]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)

In [141]:
y_train.value_counts()

Churn
0    4130
1    1495
Name: count, dtype: int64

## create the training data frame

In [142]:
df3 = X_train.copy()
df3['Churn'] = y_train

In [144]:
df3.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,...,InternetService_DSL,InternetService_Fiber optic,Contract_Month-to-month,Contract_One year,Contract_Two year,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn
684,1,0,0,0,0.000000,1,0,0,0,0,...,0,1,1,0,0,0,0,0,1,0
2446,1,0,0,0,0.239437,1,1,0,1,0,...,0,1,1,0,0,0,1,0,0,1
1680,0,0,1,1,0.774648,1,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2220,0,0,1,0,1.000000,1,0,1,1,0,...,1,0,0,0,1,1,0,0,0,0
2842,1,0,0,0,0.042254,0,0,1,0,1,...,1,0,1,0,0,0,0,0,1,0


In [145]:
df3.Churn.value_counts()

Churn
0    4130
1    1495
Name: count, dtype: int64

## split train data frame with Churn 0 and 1

In [149]:
df3_class0_maj = df3[df3.Churn==0]
df3_class1_min = df3[df3.Churn==1]

## create a function to give you the train data base on 

In [148]:
def get_train_batch(df_majority, df_minority, start, end):
    df_train = pd.concat([df_majority[start:end], df_minority], axis=0)

    X_train = df_train.drop('Churn', axis='columns')
    y_train = df_train.Churn
    return X_train, y_train

## Get the training data for 1st batch

In [150]:
X_train, y_train = get_train_batch(df3_class0_maj, df3_class1_min, 0, 1495)

In [151]:
y_preds1 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
94/94 [==============================] - 1s 1ms/step - loss: 0.6042 - accuracy: 0.6930
Epoch 2/100
94/94 [==============================] - 0s 1ms/step - loss: 0.5132 - accuracy: 0.7595
Epoch 3/100
94/94 [==============================] - 0s 1ms/step - loss: 0.4931 - accuracy: 0.7739
Epoch 4/100
94/94 [==============================] - 0s 1ms/step - loss: 0.4870 - accuracy: 0.7712
Epoch 5/100
94/94 [==============================] - 0s 959us/step - loss: 0.4826 - accuracy: 0.7732
Epoch 6/100
94/94 [==============================] - 0s 994us/step - loss: 0.4811 - accuracy: 0.7726
Epoch 7/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4794 - accuracy: 0.7756
Epoch 8/100
94/94 [==============================] - 0s 1ms/step - loss: 0.4762 - accuracy: 0.7789
Epoch 9/100
94/94 [==============================] - 0s 1ms/step - loss: 0.4751 - accuracy: 0.7763
Epoch 10/100
94/94 [==============================] - 0s 1ms/step - loss: 0.4715 - accuracy: 0.7783
Epoch

## Get the training data for 2nd batch

In [152]:
X_train, y_train = get_train_batch(df3_class0_maj, df3_class1_min, 1495, 2990)

In [153]:
y_preds2 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
94/94 [==============================] - 1s 1ms/step - loss: 0.6137 - accuracy: 0.6946
Epoch 2/100
94/94 [==============================] - 0s 1ms/step - loss: 0.5189 - accuracy: 0.7465
Epoch 3/100
94/94 [==============================] - 0s 1ms/step - loss: 0.4997 - accuracy: 0.7599
Epoch 4/100
94/94 [==============================] - 0s 1ms/step - loss: 0.4894 - accuracy: 0.7666
Epoch 5/100
94/94 [==============================] - 0s 1ms/step - loss: 0.4857 - accuracy: 0.7666
Epoch 6/100
94/94 [==============================] - 0s 986us/step - loss: 0.4822 - accuracy: 0.7696
Epoch 7/100
94/94 [==============================] - 0s 979us/step - loss: 0.4793 - accuracy: 0.7719
Epoch 8/100
94/94 [==============================] - 0s 989us/step - loss: 0.4766 - accuracy: 0.7699
Epoch 9/100
94/94 [==============================] - 0s 1ms/step - loss: 0.4765 - accuracy: 0.7699
Epoch 10/100
94/94 [==============================] - 0s 1ms/step - loss: 0.4749 - accuracy: 0.7702
Epo

## Get the training data for 3rd batch

In [155]:
X_train, y_train = get_train_batch(df3_class0_maj, df3_class1_min, 2990, 4130)

In [156]:
y_pred3 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
83/83 [==============================] - 1s 1ms/step - loss: 0.5998 - accuracy: 0.6820
Epoch 2/100
83/83 [==============================] - 0s 1ms/step - loss: 0.5055 - accuracy: 0.7594
Epoch 3/100
83/83 [==============================] - 0s 997us/step - loss: 0.4896 - accuracy: 0.7666
Epoch 4/100
83/83 [==============================] - 0s 1ms/step - loss: 0.4808 - accuracy: 0.7708
Epoch 5/100
83/83 [==============================] - 0s 1ms/step - loss: 0.4756 - accuracy: 0.7757
Epoch 6/100
83/83 [==============================] - 0s 1ms/step - loss: 0.4712 - accuracy: 0.7750
Epoch 7/100
83/83 [==============================] - 0s 1ms/step - loss: 0.4682 - accuracy: 0.7806
Epoch 8/100
83/83 [==============================] - 0s 1ms/step - loss: 0.4672 - accuracy: 0.7814
Epoch 9/100
83/83 [==============================] - 0s 1ms/step - loss: 0.4633 - accuracy: 0.7841
Epoch 10/100
83/83 [==============================] - 0s 1ms/step - loss: 0.4604 - accuracy: 0.7844
Epoch 1

In [158]:
len(y_preds1), len(y_preds2), len(y_pred3)

(1407, 1407, 1407)

## Create a final prediction out of all the three predictions made with different chunk of training data

In [166]:
y_pred_final = y_preds1.copy() # create a new numpy array by copying all the predictions of y_preds1

In [168]:
# y_pred_final the prediction of final numpy array will be now replaced by the voting amongs all other predictions
for i in range(len(y_preds1)):
    # add all the predictions if sum is 1 then majority is 0 and if its greater then 1 then majority is 1
    n_ones = y_preds1[i] + y_preds2[i] + y_pred3[i]
    if n_ones > 1:
        y_pred_final[i] = 1
    else:
        y_pred_final[i] = 0

## Get the classification report for the final prediction 

In [169]:
cl_rep = classification_report(y_test, y_pred_final)
print(cl_rep)

              precision    recall  f1-score   support

           0       0.90      0.70      0.79      1033
           1       0.49      0.78      0.60       374

    accuracy                           0.72      1407
   macro avg       0.69      0.74      0.69      1407
weighted avg       0.79      0.72      0.74      1407

